In [2]:
import requests
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

In [8]:
args = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json


In [6]:
access_token = authenticate(json['client_id'], json['client_secret'])

In [7]:
bigquery.run_query()

TypeError: BigQueryManager.run_query() missing 1 required positional argument: 'query'

In [9]:
catalog_item_id = 'MLB28017126'

url = f"https://api.mercadolibre.com/products/{catalog_item_id}/items"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
response.json()

{'paging': {'total': 7, 'offset': 0, 'limit': 100},
 'results': [{'item_id': 'MLB4212279488',
   'site_id': 'MLB',
   'seller_id': 569119547,
   'accepts_mercadopago': True,
   'price': 730.55,
   'category_id': 'MLB180269',
   'currency_id': 'BRL',
   'warranty': '',
   'condition': 'new',
   'listing_type_id': 'gold_pro',
   'international_delivery_mode': 'none',
   'tier': '',
   'inventory_id': '',
   'tags': ['good_quality_thumbnail',
    'kvs_primary',
    'extended_warranty_eligible',
    'mshops_3x_campaign',
    'immediate_payment',
    'cart_eligible'],
   'deal_ids': ['MLB24234',
    'MLB9718',
    'MLB17047',
    'MLB9711',
    'MLB33417',
    'MLB15007',
    'MLB15003',
    'MLB33463',
    'MLB13472',
    'MLB15264',
    'MLB30031',
    'MLB24527',
    'MLB24599',
    'MLB23359'],
   'official_store_id': 64562,
   'original_price': 769,
   'shipping': {'free_shipping': True,
    'store_pick_up': False,
    'mode': 'me2',
    'logistic_type': 'fulfillment',
    'tags': ['ma

In [ ]:
item_id = 'MLB4966133390'

url = f"https://api.mercadolibre.com/items/{item_id}/shipping"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
response.json()

In [8]:
item_id = 'MLB4978023790'

url = f"https://api.mercadolibre.com/items/{item_id}/shipping"

# Cabeçalhos de autorização
headers = {'Authorization': f'Bearer {access_token}'}

# Fazendo a requisição GET
response = requests.get(url, headers=headers)
data = response.json()

In [ ]:
def process_shipping(json_data):
    try:
        default_value = json_data.get('default')
        channels = json_data.get('channels', [])
        item_id = json_data.get('item_id')
        dict_list = []
        for channel in channels:
            dict_content = {
                'item_id': item_id,
                'channel_id': channel.get('id'),
                'mode': channel.get('mode'),
                'logistic_type': channel.get('logistic_type'),
                'local_pick_up': channel.get('local_pick_up'),
                'free_shipping': channel.get('free_shipping'),
                'store_pick_up': channel.get('store_pick_up'),
                'default': default_value
            }
            dict_list.append(dict_content)
        return dict_list
    except Exception as e:
        print(f'Error processing json: {json_data}, error: {e}')
        return []
import pandas as pd
pd.DataFrame(process_shipping(data))

In [ ]:
data = json
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_FULLFILMENT
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/catelog_details/date={today_str}/'
items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names='inventory_id'
)

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json


def insert_bq_competitors_prices(request):

    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_CATALOG_COMPETITORS
    blob_shipping_cost = settings.BLOB_COMPETITORS_CATALOG(store_name)

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Get dates to treat
    list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)

    print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')

    df_processed_data = pd.DataFrame()

    for date in list_dates_to_process:

        # Transform date to string
        date_to_process = date.strftime('%Y-%m-%d')
        print(f'Processing date: {date_to_process}')
        # Get blob with the date
        blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
        # List all the files
        blobs = storage.list_blobs(bucket_name, blob_prefix)

        # Processing each blob
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)

            for json in content:
                processed_dict = process_prices(json)

                if isinstance(processed_dict, list):
                    df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
                else:
                    continue

        df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
        df_processed_data['process_time'] = datetime.now()
        df_processed_data['seller_id'] = seller_id

        print(f'*** Finished treating all data. {df_processed_data.shape[0]} products ***')

        print('** Deleting existing data **')
        bigquery.delete_existing_data(destiny_table, seller_id, date_to_process)
        
        print('** Correct dataframe schema **')
        bigquery.match_dataframe_schema(df_processed_data, destiny_table)

        print('** Inserting data into BQ**')
        bigquery.insert_dataframe(df_processed_data, destiny_table)

        print('** Updating log table **')
        bigquery.update_logs_table(seller_id, date_to_process, destiny_table, table_management)

    return ('Success', 200)


def process_prices(json):

    try:
        extracted_data = []
        # Dicionário temporário para priorizar os preços por canal
        price_by_channel = {}
        for price in json['prices']:
            channel = price['conditions']['context_restrictions']
            if len(channel) == 1:
                channel = channel[0]

                # Se ainda não há preço para o canal ou se o preço atual é promoção, atualiza
                if channel not in price_by_channel or price['type'] == 'promotion':
                    price_by_channel[channel] = {
                        'item_id': json.get('id'),
                        'price_id': price.get('id'),
                        'regular_amount': price.get('regular_amount'),
                        'price': price.get('amount'),
                        'channel': channel,
                        'last_updated': price.get('last_updated')
                    }
        # Converte os valores armazenados para uma lista
        extracted_data.extend(price_by_channel.values())

        return extracted_data
    
    except:
        print(f'Error processing json: {json}')
        

In [2]:
json = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [ ]:
data = json
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_CATALOG_COMPETITORS
blob_shipping_cost = settings.BLOB_COMPETITORS_CATALOG(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)
    # Processing each blob
    for blob in blobs:
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)
        for json in content:
            processed_dict = process_competitors_catalog(json)
            if isinstance(processed_dict, list):
                df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
            else:
                continue
    df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
    df_processed_data['process_time'] = datetime.now()
    df_processed_data['seller_id'] = seller_id

In [ ]:
df_processed_data

In [ ]:
content[0]['item_id']

In [ ]:
data = content[0]['results']
data[0]

In [ ]:
content[0]['results'][0].get('category_id')

In [ ]:
list_proc = process_competitors_catalog(content[0])
pd.DataFrame(list_proc)

In [28]:
def process_competitors_catalog(json):

    catalog_id = json['item_id']
    results_list = []  # Create an empty list to store the dictionaries

    try:
        for item in json['results']:
            dict_content = {
                'catalog_product_id': catalog_id, 
                'item_id' : item.get('item_id'),
                'competitors_type': 'catalog',
                'category_id': item.get('category_id'),
                'official_store_id': item.get('official_store_id'),
                'competitor_seller_id': item.get('seller_id'),
                'listing_type_id': item.get('listing_type_id'),
                'condition': item.get('condition'),
            }
            
            results_list.append(dict_content)  # Append each dictionary to the list
        
        return results_list  # Return the full list after iterating through all items
    
    except Exception as e:
        print(f'Error processing json: {json}. Error: {str(e)}')
        return None  # Optionally return None if there's an error


In [2]:
# Improve costs
data = json = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [3]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime
import requests

semaphore = asyncio.Semaphore(100)  # Control the number of simultaneous requests


# Parsing request data
# data = request.get_json()
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_COSTS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Getting params to see costs
query = f'''
    with items_details as (
    select distinct
        item_id,
        listing_type,
        category_id
    from datalake-v2-424516.datalake_v2.items_details
    where
        1=1
        and date(correspondent_date) = current_date()
        and seller_id = {seller_id}
    )
    select 
    p.item_id as id,
    d.listing_type as listing_type_id,
    d.category_id,
    p.price,
    p.channel
    from datalake-v2-424516.datalake_v2.items_prices p
    inner join items_details d
    on p.item_id = d.item_id
    where 
        1=1
        and date(p.correspondent_date) = current_date()
        and channel is not null
'''
# blob_items_prefix = f'{store_name}/meli/api_response/item_detail/date={today_str}/'
# items_id = fetch_items_from_storage(
# storage, 
# bucket_name, 
# blob_items_prefix, 
# key_names=['id','price', 'category_id', 'listing_type_id']
# )

df_params = bigquery.run_query(query)
# items = df_params[['id','channel']].to_dict(orient='records')
df_params['channel'] = df_params['channel'].apply(lambda x : x.replace('channel_', '')).drop(columns = 'channel')
items_id = df_params.to_dict(orient='records')

print(f'** Items found: {len(items_id)}**')
print(f'** Cleaning blob **')
# Path for saving 
blob_basic_path = settings.BLOB_COSTS(store_name)
date_blob_path = f'{blob_basic_path}date={today_str}/'
# Clean existing files in the storage bucket
storage.clean_blobs(bucket_name, date_blob_path)
print(f'** Starting API requests for {len(items_id)} items**')
# URL function for API
url = settings.URL_COST
headers = {'Authorization': f'Bearer {access_token}'}

# Batch processing the API requests
async with aiohttp.ClientSession() as session:
    await batch_process(session, items_id, url, headers, 
                        bucket_name, date_blob_path, storage, 
                        params = items_id, add_item_id = True)
    
log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)



** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
** Items found: 1976**
** Cleaning blob **
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_0__process_time=2024-10-15T14:29:29.211919-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_10__process_time=2024-10-15T14:30:30.258868-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_11__process_time=2024-10-15T14:30:30.304502-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_12__process_time=2024-10-15T14:30:30.257919-03:00.json
Deleting blob: hubsmarthome/meli/api_response/item_cost/date=2024-10-15/batch_13__process_time=2024-10-15T14:30:30.346773-0

In [4]:
log_process(seller_id, destiny_table, today_str, table_management, processed_to_bq=False)


# Visitas

In [1]:
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.common.utils import batch_process, log_process, authenticate, fetch_items_from_storage
from src.config import settings
import json
import asyncio
import aiohttp
from datetime import datetime

semaphore = asyncio.Semaphore(100)  # Control the number of simultaneous requests

async def main_async(request):
    # Parsing request data
    data = request.get_json()
    client_id = data.get('client_id')
    client_secret = data.get('client_secret')
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')
    access_token = data.get('access_token')

    print('** Defining authentication... **')
    # Authenticate (assuming this is now centralized in utils.py or a similar file)
    if not access_token:
        access_token = authenticate(client_id, client_secret)  # You can add this to a common module

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_VISITS

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')
    
    # Fetch item IDs from the storage bucket
    blob_items_prefix = f'{store_name}/meli/api_response/items/date={today_str}/'
    items_id = fetch_items_from_storage(
    storage, 
    bucket_name, 
    blob_items_prefix, 
    key_names='results'
    )

    print(f'** Items found: {len(items_id)}**')

    print(f'** Cleaning blob **')
    # Path for saving 
    blob_basic_path = settings.BLOB_VISITS(store_name)
    date_blob_path = f'{blob_basic_path}date={today_str}/'

    # Clean existing files in the storage bucket
    storage.clean_blobs(bucket_name, date_blob_path)

    print(f'** Starting API requests for {len(items_id)} items**')
    # URL function for API
    url = settings.URL_ITEM_DETAIL

    return 

In [2]:
args = {
  "access_token": None,
  "client_id": "4959083987776428",
  "client_secret": "Hw9wWSydd8PMvMEJewWoMvKGYMAWyKEw",
  "seller_id": 189643563,
  "store_name": "hubsmarthome"
}

In [4]:
data = args
client_id = data.get('client_id')
client_secret = data.get('client_secret')
store_name = data.get('store_name')
seller_id = data.get('seller_id')
access_token = data.get('access_token')
print('** Defining authentication... **')
# Authenticate (assuming this is now centralized in utils.py or a similar file)
if not access_token:
    access_token = authenticate(client_id, client_secret)  # You can add this to a common module
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_VISITS
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')

# Fetch item IDs from the storage bucket
blob_items_prefix = f'{store_name}/meli/api_response/items/date={today_str}/'
items_id = fetch_items_from_storage(
storage, 
bucket_name, 
blob_items_prefix, 
key_names='results'
)
print(f'** Items found: {len(items_id)}**')
print(f'** Cleaning blob **')
# Path for saving 
blob_basic_path = settings.BLOB_VISITS(store_name)
date_blob_path = f'{blob_basic_path}date={today_str}/'
# Clean existing files in the storage bucket
storage.clean_blobs(bucket_name, date_blob_path)
print(f'** Starting API requests for {len(items_id)} items**')
# URL function for API
url = settings.URL_ITEM_DETAIL

** Defining authentication... **
** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Reading file: hubsmarthome/meli/api_response/items/date=2024-10-16/total_items=991__processing-date=2024-10-16T07:00:00.230851-03:00.json
** Items found: 991**
** Cleaning blob **
All blobs with prefix hubsmarthome/meli/api_response/item_visits/date=2024-10-16/ have been deleted.
** Starting API requests for 991 items**


In [6]:
def visits_to_dataframe(json_visit):
    # Initialize lists to store the extracted data
    
    item_id = json_visit['item_id']
    visits_data = json_visit['results']
    dates = []
    total_visits = []
    companies = []

    # Iterate through the data
    for visit in visits_data:
        dates.append(visit['date'])
        total_visits.append(visit['total'])
        company_list = [detail['company'] for detail in visit['visits_detail']]
        companies.append(", ".join(company_list))  # Join company names if there are multiple
    
    # Create a DataFrame
    df = pd.DataFrame({
        'item_id':item_id,
        'date': dates,
        'total_visits': total_visits,
        'companies': companies
    })
    
    return df


In [34]:
url = lambda item_id : f'https://api.mercadolibre.com/items/{item_id}/visits/time_window?last=1&unit=day'
headers = {'Authorization': f'Bearer {access_token}'}

item = 'MLB3326162963'
response = requests.get(url(item), headers=headers)
response.json()

{'item_id': 'MLB3326162963',
 'date_from': '2024-10-15T00:00:00Z',
 'date_to': '2024-10-16T00:00:00Z',
 'total_visits': 162,
 'last': 1,
 'unit': 'day',
 'results': [{'date': '2024-10-15T00:00:00Z',
   'total': 86,
   'visits_detail': [{'company': 'mercadolibre', 'quantity': 86}]},
  {'date': '2024-10-16T00:00:00Z',
   'total': 76,
   'visits_detail': [{'company': 'mercadolibre', 'quantity': 76}]}]}

In [40]:
blob_basic_path = settings.BLOB_VISITS(store_name)
bool_first_time = storage.blob_exists(bucket_name, blob_basic_path)
bool_first_time

False

In [5]:
from tqdm import tqdm

# access_token = authenticate(client_id, client_secret)

url = lambda item_id : f'https://api.mercadolibre.com/items/{item_id}/visits/time_window?last=150&unit=day&ending=2024-09-07'
headers = {'Authorization': f'Bearer {access_token}'}

df_visitas = pd.DataFrame()

for i, item in tqdm(enumerate(items_id)):
    
    response = requests.get(url(item), headers=headers)
    print(response.status_code)
    daily_visits = response.json()
    
    df_ = visits_to_dataframe(daily_visits)
    
    df_visitas = pd.concat([df_visitas, df_], ignore_index=True)
    
    if i % 50 == 0:
        print('Pause')
        time.sleep(15)


['MLB4187240120',
 'MLB4583731462',
 'MLB3360473319',
 'MLB3502724349',
 'MLB5009741936',
 'MLB2668403665',
 'MLB3815017045',
 'MLB5057931704',
 'MLB3812753195',
 'MLB4169384508',
 'MLB3326182913',
 'MLB4267999730',
 'MLB4988969662',
 'MLB5058056958',
 'MLB4681850534',
 'MLB5057897718',
 'MLB3788298581',
 'MLB3816696423',
 'MLB4681887684',
 'MLB4525890202',
 'MLB3326149813',
 'MLB3865441845',
 'MLB3635530579',
 'MLB3847735843',
 'MLB4507378594',
 'MLB3752786754',
 'MLB3759173609',
 'MLB5057919064',
 'MLB3400456361',
 'MLB5009741612',
 'MLB3865393145',
 'MLB3817731603',
 'MLB3816710629',
 'MLB3204690444',
 'MLB3655026323',
 'MLB4600690612',
 'MLB3609380433',
 'MLB3865368457',
 'MLB4149295454',
 'MLB4183813096',
 'MLB5057896886',
 'MLB3530787137',
 'MLB3844267265',
 'MLB4601135750',
 'MLB3627508759',
 'MLB3764961583',
 'MLB4945145142',
 'MLB3817765517',
 'MLB3844420753',
 'MLB3865417333',
 'MLB5057955364',
 'MLB3865322717',
 'MLB3811857185',
 'MLB3819388357',
 'MLB3844458369',
 'MLB28229

In [4]:
import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from src.common.cloud_storage_connector import CloudStorage
from src.common.bigquery_connector import BigQueryManager
from src.config import settings
import json


def insert_bq_visits(request):

    data = request.get_json()
    store_name = data.get('store_name')
    seller_id = data.get('seller_id')

    print('** Connecting to storage and BigQuery... **')
    # Initialize storage and BigQuery
    storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
    bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)

    # Define paths and table names from the config
    bucket_name = settings.BUCKET_STORES
    table_management = settings.TABLE_MANAGEMENT
    destiny_table = settings.TABLE_VISITS
    blob_shipping_cost = settings.BLOB_VISITS(store_name)

    # Define today's date
    today_str = datetime.today().strftime('%Y-%m-%d')

    # Get dates to treat
    list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)

    print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')

    df_processed_data = pd.DataFrame()

    for date in list_dates_to_process:

        # Transform date to string
        date_to_process = date.strftime('%Y-%m-%d')
        print(f'Processing date: {date_to_process}')
        # Get blob with the date
        blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
        # List all the files
        blobs = storage.list_blobs(bucket_name, blob_prefix)

        # Processing each blob
        for blob in blobs:
            print(f"Reading file: {blob.name}")
            content = storage.download_json(bucket_name, blob.name)

            for json in content:
                processed_dict = process_shipping(json)

                if isinstance(processed_dict, list):
                    df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
                else:
                    continue

        df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
        df_processed_data['process_time'] = datetime.now()
        df_processed_data['seller_id'] = seller_id

        print(f'*** Finished treating all data. {df_processed_data.shape[0]} products ***')

        print('** Deleting existing data **')
        bigquery.delete_existing_data(destiny_table, seller_id, date_to_process)
        
        print('** Correct dataframe schema **')
        bigquery.match_dataframe_schema(df_processed_data, destiny_table)

        print('** Inserting data into BQ**')
        bigquery.insert_dataframe(df_processed_data, destiny_table)

        print('** Updating log table **')
        bigquery.update_logs_table(seller_id, date_to_process, destiny_table, table_management)

    return ('Success', 200)

def process_shipping(json_data):
    try:
        default_value = json_data.get('default')
        channels = json_data.get('channels', [])
        item_id = json_data.get('item_id')
        dict_list = []
        for channel in channels:
            dict_content = {
                'item_id': item_id,
                'channel_id': channel.get('id'),
                'mode': channel.get('mode'),
                'logistic_type': channel.get('logistic_type'),
                'local_pick_up': channel.get('local_pick_up'),
                'free_shipping': channel.get('free_shipping'),
                'store_pick_up': channel.get('store_pick_up'),
                'default_shipping': default_value
            }
            dict_list.append(dict_content)
        return dict_list
    except Exception as e:
        print(f'Error processing json: {json_data}, error: {e}')
        return []

        
                        



In [7]:
json

<module 'json' from 'c:\\Users\\User\\anaconda3\\lib\\json\\__init__.py'>

In [16]:
data=args
store_name = data.get('store_name')
seller_id = data.get('seller_id')
print('** Connecting to storage and BigQuery... **')
# Initialize storage and BigQuery
storage = CloudStorage(credentials_path=settings.PATH_SERVICE_ACCOUNT)
bigquery = BigQueryManager(credentials_path=settings.PATH_SERVICE_ACCOUNT)
# Define paths and table names from the config
bucket_name = settings.BUCKET_STORES
table_management = settings.TABLE_MANAGEMENT
destiny_table = settings.TABLE_VISITS
blob_shipping_cost = settings.BLOB_VISITS(store_name)
# Define today's date
today_str = datetime.today().strftime('%Y-%m-%d')
# Get dates to treat
list_dates_to_process = bigquery.get_list_dates_to_process(seller_id, table_management, destiny_table)
print(f'*** Starting to process dates: {len(list_dates_to_process)} dates to process  ***')
df_processed_data = pd.DataFrame()
for date in list_dates_to_process:
    # Transform date to string
    date_to_process = date.strftime('%Y-%m-%d')
    print(f'Processing date: {date_to_process}')
    # Get blob with the date
    blob_prefix = blob_shipping_cost + f'date={date_to_process}/'
    # List all the files
    blobs = storage.list_blobs(bucket_name, blob_prefix)
    # Processing each blob
    for blob in blobs:
        print(f"Reading file: {blob.name}")
        content = storage.download_json(bucket_name, blob.name)
        for json in content:
            processed_dict = process_visits(json)
            if isinstance(processed_dict, list):
                df_processed_data = pd.concat([df_processed_data, pd.DataFrame(processed_dict)], ignore_index = True)
            else:
                continue

    df_processed_data['correspondent_date'] = pd.to_datetime(date_to_process)
    df_processed_data['process_time'] = datetime.now()
    df_processed_data['seller_id'] = seller_id

** Connecting to storage and BigQuery... **
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
Using local credentials from: C:/Users/User/Documents/papa preco/service account/service_account_datalakev2.json
*** Starting to process dates: 1 dates to process  ***
Processing date: 2024-10-16
Reading file: hubsmarthome/meli/api_response/item_visits/date=2024-10-16/batch_0__process_time=2024-10-16T22:43:43.547322-03:00.json
Reading file: hubsmarthome/meli/api_response/item_visits/date=2024-10-16/batch_1__process_time=2024-10-16T22:43:43.255893-03:00.json
Reading file: hubsmarthome/meli/api_response/item_visits/date=2024-10-16/batch_2__process_time=2024-10-16T22:44:44.585362-03:00.json
Reading file: hubsmarthome/meli/api_response/item_visits/date=2024-10-16/batch_3__process_time=2024-10-16T22:44:44.032510-03:00.json
Reading file: hubsmarthome/meli/api_response/item_visits/date=2024-10-16/batch_4__process_time=2024-10-16T22:45:45.

In [17]:
df_processed_data

,item_id,num_visits,date,correspondent_date,process_time,seller_id
0,MLB3620576243,0,2024-10-15T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
1,MLB3690852072,0,2024-10-15T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
2,MLB4183845176,5,2024-10-15T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
3,MLB4183845176,11,2024-10-16T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
4,MLB4969934040,3,2024-10-15T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
...,...,...,...,...,...,...
1257,MLB3764961583,19,2024-10-15T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
1258,MLB3764961583,5,2024-10-16T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
1259,MLB3819376427,0,2024-10-15T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563
1260,MLB4583731462,16,2024-10-15T00:00:00Z,2024-10-16,2024-10-17 18:34:00.641601,189643563


In [12]:
data = content[0]
data

{'item_id': 'MLB4080730664',
 'date_from': '2024-10-15T00:00:00Z',
 'date_to': '2024-10-16T00:00:00Z',
 'total_visits': 12,
 'last': 1,
 'unit': 'day',
 'results': [{'date': '2024-10-15T00:00:00Z',
   'total': 8,
   'visits_detail': [{'company': 'mercadolibre', 'quantity': 8}]},
  {'date': '2024-10-16T00:00:00Z',
   'total': 4,
   'visits_detail': [{'company': 'mercadolibre', 'quantity': 4}]}]}

In [19]:
def process_visits(json_data):

    try:
        item_id = json_data.get("item_id")
        list_visits = []
        for visits_per_date in json_data.get('results',[]):

            dict_content = {
                "item_id": item_id,
                "num_visits": visits_per_date.get('total'),
                "date": visits_per_date.get('date')
            }

            list_visits.append(dict_content)

        return list_visits

    except Exception as e:
        print(f'Error processing json: {json_data}, error: {e}')
        return []